# Combining DataV2 and DataV3 into Dataset_XML

This notebook combines the XML-based license plate datasets (DataV2 and DataV3) into a single unified dataset called Dataset_XML.

Both datasets use the same annotation format with bounding boxes defined as:
- `xmin, ymin`: Top-left corner
- `xmax, ymax`: Bottom-right corner